In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

In [3]:
with initialize(config_path="../configs"):
    cfg = compose(config_name='train', overrides=['module=yolov5.yaml'])
    print(OmegaConf.to_yaml(cfg))

seed: 42
name: null
trainer:
  _target_: pytorch_lightning.Trainer
  gpus: 1
  accumulate_grad_batches: 4
module:
  model_type: yolov5
  backbone: large_p6
  img_size:
  - 960
  - 768
  model:
    _target_: src.modules.yolov5.YoloV5Module
    model:
      _target_: src.models.yolov5.model
      backbone:
        _target_: src.models.yolov5.backbones.${module.backbone}
        pretrained: 0
      num_classes: 2
      img_size: 960
    learning_rate: 0.0001
    weight_decay: 1.0e-06
datamodule:
  _target_: src.datamodules.dental_caries.DentalCaries
  data_root: /datagrid/personal/kuntluka/dental_merged
  model_type: ${module.model_type}
  ann_file: annotations2.json
  batch_size: 1
  num_workers: 8
  limit_train_samples: null
  train_val_test_split:
  - 0.7
  - 0.15
  - 0.15
  seed: ${seed}
logger:
  wandb:
    _target_: pytorch_lightning.loggers.wandb.WandbLogger
    project: MT2
    offline: false
    id: null
    log_model: false
    prefix: ''
    job_type: train
    group: ''
    ta

In [4]:
# import src.modules.yolov5
model = hydra.utils.instantiate(cfg.module.model)
# model._get_lightning_module_state_dict(),


/home/kuntik/miniconda3/envs/det/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


ImportError: Error loading module 'src.modules.yolov5.YoloV5Module'